In [12]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -Uq datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
)
import warnings

warnings.filterwarnings("ignore")

In [14]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="fka/awesome-chatgpt-prompts", filename="prompts.csv",repo_type="dataset")

'/root/.cache/huggingface/hub/datasets--fka--awesome-chatgpt-prompts/snapshots/7baf3f8a5f3d38acc585d42d12193b27baf8cf79/prompts.csv'

In [15]:
import pandas as pd
df = pd.read_csv("/root/.cache/huggingface/hub/datasets--fka--awesome-chatgpt-prompts/snapshots/7baf3f8a5f3d38acc585d42d12193b27baf8cf79/prompts.csv")
df.head()

,act,prompt
0,Linux Terminal,I want you to act as a linux terminal. I will ...
1,English Translator and Improver,"I want you to act as an English translator, sp..."
2,`position` Interviewer,I want you to act as an interviewer. I will be...
3,JavaScript Console,I want you to act as a javascript console. I w...
4,Excel Sheet,I want you to act as a text based excel. you'l...


In [16]:
df['text'] = f"""Below is an instruction that describes a task, Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
I want to make chatgpt act as a {df["act"]}
### Response:
{df["prompt"]}"""

In [17]:
df.head()

,act,prompt,text
0,Linux Terminal,I want you to act as a linux terminal. I will ...,"Below is an instruction that describes a task,..."
1,English Translator and Improver,"I want you to act as an English translator, sp...","Below is an instruction that describes a task,..."
2,`position` Interviewer,I want you to act as an interviewer. I will be...,"Below is an instruction that describes a task,..."
3,JavaScript Console,I want you to act as a javascript console. I w...,"Below is an instruction that describes a task,..."
4,Excel Sheet,I want you to act as a text based excel. you'l...,"Below is an instruction that describes a task,..."


In [18]:
!pip install datasets

In [19]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [20]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
)

model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [22]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [23]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 50
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 300
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [24]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

In [25]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [33]:
trainer.train()

Step,Training Loss
10,0.080000
20,0.076400
30,0.074700
40,0.077300
50,0.074200
60,0.076500
70,0.074300


TrainOutput(global_step=75, training_loss=0.07589558521906535, metrics={'train_runtime': 789.8772, 'train_samples_per_second': 1.519, 'train_steps_per_second': 0.095, 'total_flos': 5423714408374272.0, 'train_loss': 0.07589558521906535, 'epoch': 7.69})

In [36]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [38]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [39]:
model.push_to_hub("Veer15/llama-gpt-prompter",create_pr=1)

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Veer15/llama-gpt-prompter/commit/ccf743c729dda2dea0749834f2a96e1ab3c3b052', commit_message='Upload model', commit_description='', oid='ccf743c729dda2dea0749834f2a96e1ab3c3b052', pr_url='https://huggingface.co/Veer15/llama-gpt-prompter/discussions/1', pr_revision='refs/pr/1', pr_num=1)